In [63]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports

In [64]:
import os
import pandas as pd

import numpy as np
import jax
jax.config.update('jax_platform_name', 'gpu')



In [65]:
fn_EcoCyc = '../data/sRNA/EcoCyc/EcoCyc_regulatory_RNAs.csv'
ecocyc = pd.read_csv(fn_EcoCyc)
fn_RNAInter = os.path.join('..', 'data', 'sRNA', 'RNAInter', 'Download_data_RR.csv')
rnainter = pd.read_csv(fn_RNAInter)
# data = pd.concat([pd.read_csv(fn_RNAInter), pd.read_csv(fn_EcoCyc)])
try:
    rnainter = rnainter.drop(columns=['Unnamed: 0'])
except:
    pass


### Start with RNAInter

In [102]:
data = pd.DataFrame()
rnainter['Database'] = 'RNAInter'
for n, o in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database'], ['Interactor1.Symbol', 'Raw_ID1', 'Category1', 'Interactor1.Symbol', 'Sequence1', 'Database']):
    data[n] = rnainter.drop_duplicates(subset=['Interactor1.Symbol'])[o]

data = pd.concat([data, rnainter.drop_duplicates(subset=['Interactor2.Symbol']).reset_index()[['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Sequence2']].rename(
    columns={j: i for i, j in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence'], ['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Interactor2.Symbol', 'Sequence2'])})], axis=0)

data = data[~data['Name'].isna()]
data = data.drop_duplicates(subset=['Name'])

In [103]:
data['Symbol'].value_counts()


Symbol
acnA    1
metH    1
malK    1
manX    1
manY    1
       ..
fumC    1
fur     1
gadB    1
gadX    1
symE    1
Name: count, Length: 154, dtype: int64

### Now EcoCyc

In [104]:
ecocyc.head()

,Ontology - all instances of class from My Favorites,Matches,Common-Name,GO-Terms,Names,Object ID,Gene,Sequence - DNA sequence,Copy-Number,Common-Name.1,Category,Database
0,6S-RNA,Regulatory-RNAs,6S RNA,NaN,6S RNA // SsrS // Ssr,6S-RNA,EG30099,ATTTCTCTGAGATGTTCGCAAGCGGGCCAGTCCCCTGAGCCGATAT...,NaN,ssrS,sRNA,EcoCyc
1,RDLD-RNA,Regulatory-RNAs,antisense regulatory RNA RdlD,GO:2000766 // GO:0030371,antisense regulatory RNA RdlD // RdlD // RdlD ...,RDLD-RNA,G0-9042,GTCTAGAGTCAAGATTAGCCCCCGTGGTGTTGTCAGGTGCATACCT...,NaN,rdlD,sRNA,EcoCyc
2,RNA0-441,Regulatory-RNAs,<i>arfA</i>-derived small regulatory RNA,GO:0006970 // GO:0003729,<i>arfA</i>-derived small regulatory RNA,RNA0-441,G0-17130,AAGAAAGTAAATCACTTTTTTACCACTGGTCTTCTGCTTTCAGGCG...,NaN,yrdG,sRNA,EcoCyc
3,RNA0-409,Regulatory-RNAs,e14 prophage; small regulatory RNA C0293,GO:0035194 // GO:0040033 // GO:0003729,e14 prophage; small regulatory RNA C0293 // C0293,RNA0-409,G0-8897,TACTCACCCGGGACTCGCCAGGGGACAGCCAACAGGCATTGGGTGC...,NaN,C0293,sRNA,EcoCyc
4,RNA0-162,Regulatory-RNAs,putative antisense regulatory RNA RdlB,GO:1902201,putative antisense regulatory RNA RdlB // RdlB,RNA0-162,G0-9606,GTCTGGTTTCAAGATTAGCCCCCGTTCTGTTGTCAGGTTTTACCTC...,NaN,rdlB,sRNA,EcoCyc


In [105]:
ecocyc['Category'] = 'sRNA'
ecocyc['Database'] = 'EcoCyc'
ecocyc = ecocyc.drop_duplicates(subset=['Common-Name'])

data = pd.concat([data, ecocyc.rename(columns={j: i for i, j in zip(['Name', 'ID', 'Symbol', 'Sequence'], ['Common-Name', 'Object ID', 'Common-Name.1', 'Sequence - DNA sequence'])})[
    ['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database']]], axis=0)

# Delete duplicates responsibly

In [110]:
doubles = data['Symbol'].value_counts().index.to_list()

for d in doubles:
    
    if data[data['Symbol'] == d]['Sequence']

In [93]:
data[data['Symbol'] == 'ryhB']

,Name,ID,Category,Symbol,Sequence,Database
329,ryhB,NCBI:2847761,sRNA,ryhB,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...,RNAInter
0,NaN,NCBI:2847761,sRNA,ryhB,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...,NaN
63,small regulatory RNA RyhB,SRAI-RNA,sRNA,ryhB,GCGATCAGGAAGACCCTCGCGGAGAACCTGAAAGCACGACATTGCT...,EcoCyc


In [94]:
data.drop_duplicates(subset=['Symbol'])

,Name,ID,Category,Symbol,Sequence,Database
0,acnA,NCBI:946724,mRNA,acnA,ATTCGGAACGAGGCCTGAAGCAGTGTCGCCGTCCCTCTGCCTTGCA...,RNAInter
1,acrZ,NCBI:945365,mRNA,acrZ,ACTTACTACTGTCTTCGGGGGGTCCGAGGTTTCTGGGGGGTCGTAC...,RNAInter
2,arcZ,NCBI:2847690,sRNA,arcZ,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,RNAInter
4,argR,NCBI:947861,mRNA,argR,GACAATGGCGATAGTATGGCGGTTGTTTCTTTCCCATCTCTACTCA...,RNAInter
5,arrS,NCBI:11115378,sRNA,arrS,CTCTCCCTCTCTTTCTCTCTTCTCCGCGGCGATACGATCCGAGATG...,RNAInter
...,...,...,...,...,...,...
70,small regulatory RNA SroC,RNA0-124,sRNA,sroC,ACTAATTACAAGAACCAGGGGCGGAAATTCCAGCCCTCTCGATTGT...,EcoCyc
71,small regulatory RNA ZbiJ,RNA0-417,sRNA,zbiJ,TTTCCCTGTCTGTTTGCCGACAGACGCATATGCTCTAACCCTCATT...,EcoCyc
72,small RNA SibB,RYED-RNA,sRNA,sibB,GAGGGTAGAGCGGGGTTTCCCCCGCCCTGGTAGTCTTAGTAAGCGG...,EcoCyc
73,small RNA SibD,C0730-RNA,sRNA,sibD,ACAAGGGTGAGGGAGGATTTCTCCCCCCTCTGATTGGCTGTTAATA...,EcoCyc
